In [2]:
# Import required libraries
import os
from pathlib import Path
import re
import pybedtools
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
from collections import defaultdict
import pycircos

Garc = pycircos.Garc
Gcircle = pycircos.Gcircle


In [3]:
# Define a dictionary to store DMR paths
patient_dmrs = {}

# Base directories
input_directory = Path("path/to/input/directory") # Change this to your directory
results_directory = Path("path/to/output/directory") # Change this to your directory

# Subdirectories
dmr_directory = input_directory/"DMR_files"
cpg_directory = input_directory/"CpG_files"
tss_directory = input_directory/"TSS_files"
upwards_methylated_output_directory = results_directory/"Upwards_methylated"
downwards_methylated_output_directory = results_directory/"Downwards_methylated"
bed_output_directory = results_directory/"BED_files"
cpg_and_tss_output_directory = results_directory/"Corrected_CpGIs_"

# Columns to keep in the output DMR CSV files
columns_to_keep = ['Chromosome', 'Start', 'End', 'SiteCount', 'Length', 'Percentage', 'FoldChange']

# Create the output directories if they don't exist
directories_to_create = [
    results_directory,
    upwards_methylated_output_directory,
    downwards_methylated_output_directory,
    bed_output_directory,
    cpg_and_tss_output_directory
]

for directory in directories_to_create:
    os.makedirs(directory, exist_ok=True)

In [4]:
# Define the transformation function that corrects the original DMR files if the have any format flaws
def transform_row(row):
    # Check if Length is negative
    if row['Length'] < 0:
        # Make Length positive
        row['Length'] = abs(row['Length'])
        row['End'], row['Length'] = row['Length'], row['End']
        row['Start'], row['End'] = row['End'], row['Start']
    
    return row

# Function to short chromomosomes in the format 1,2,3...X,Y
def chrom_sorter(chrom):
    chrom = chrom[3:]  # Remove 'chr' prefix
    if chrom.isdigit():  # For numeric chromosomes
        return int(chrom)
    elif chrom == 'X':
        return 23
    elif chrom == 'Y':
        return 24
    else:
        return 25  # For any non-standard chromosome identifiers, sort them at the end

In [7]:
# Iterate over each DMR file in the input directory
for filename in os.listdir(dmr_directory):
    if filename.endswith('.csv'):
        # Extract patient ID from the filename using regular expression
        match = re.search(r'([NR|R]+)_(AZA|DAC)_(\d+)', filename)
        if match:
            patient_id = match.group(1)
            treatment = match.group(2)
        else:
            # Handle cases where the pattern doesn't match
            print(f"Could not extract patient ID and treatment from the filename: {filename}")
            continue
        
        # Specify input and output file paths
        input_file_path = dmr_directory/filename
        output_file_path = results_directory/filename

        upwards_methylated_output_file_path = upwards_methylated_output_directory/f"{os.path.splitext(filename)[0]}_upwards.csv"
        downwards_methylated_output_file_path = downwards_methylated_output_directory/f"{os.path.splitext(filename)[0]}_downwards.csv"

        patient_dmrs[patient_id] = (output_file_path, upwards_methylated_output_file_path, downwards_methylated_output_file_path)

        # Read the CSV file using pandas
        df = pd.read_csv(input_file_path, delimiter=',', low_memory = False)

        # Transform the rows
        df = df.apply(transform_row, axis=1)

        # Select specified columns and drop duplicates
        df = df[columns_to_keep].drop_duplicates()
        
        # Convert percentage to float.
        df['Percentage'] = df['Percentage'].astype(float)  # Replace commas with dots and convert to float

        # Rename columns
        df = df.rename(columns={
            'Chromosome': 'chrom',
            'Start': 'chromStart',
            'End': 'chromEnd',
            'SiteCount': 'LpnPI-sites',
            'Percentage': 'methRatio',
        })

        # Log transform the methRatio column
        df['log10(methRatio)'] = np.log10(df['methRatio'])

        # Sort chromosomes in the order of chr1, chr2 ,chr3 and so on.
        df['chromSort'] = df['chrom'].apply(chrom_sorter)
        df = df.sort_values(by=['chromSort', 'chromStart']).drop('chromSort', axis=1)

        # Apply filters to keep only regions with FoldChange >=2, meaning either an increase or decrease in methylation level by at least 100%.
        # Also keep regions that have at least 20 LpnPI-sites, indicating significant methylation or demethylation.
        df = df[(df['FoldChange'] >= 2) & (df['LpnPI-sites'] >= 20)]

        # Remove chromosomes of alternative and random contig from the dataset.
        df = df[df['chrom'].str.match(r'^chr((\d{1,2})|(X|Y))$')]
        df = df.drop(columns=['LpnPI-sites'])  # Drop the intermediate columns

        # Split data into upwards and downwards methylated regions.
        upwards_meth_df = df[df['log10(methRatio)'] > 0]
        downwards_meth_df = df[df['log10(methRatio)'] < 0]

        # Save files to csv.
        upwards_meth_df.to_csv(upwards_methylated_output_file_path, index=False, sep=',')
        downwards_meth_df.to_csv(downwards_methylated_output_file_path, index=False, sep=',')
        df.to_csv(output_file_path, index=False, sep=',')

In [9]:
# Create lists to group the CpGIs and TSS files for the 2 patients
patient_6281 = [cpg_directory/"6281_CpG.csv", tss_directory/"6281_TSS.csv"]
patient_5889 = [cpg_directory/"5889_CpG.csv", tss_directory/"5889_TSS.csv"]

def format_file(input_file, output_directory):
    # Read file into a pandas dataframe
    df = pd.read_csv(input_file, low_memory=False)

    # Rename columns early in the process
    df = df.rename(columns={
        'Chromosome': 'chrom',
        'Start': 'chromStart',
        'End': 'chromEnd',
        'Percentage(Y/X)': 'methRatio',
        'log10Score': 'log10(methRatio)'
    })

    # Convert the q-value column to numeric format and filter out rows with q-value higher than 0.05
    df['q-value'] = pd.to_numeric(df['q-value'], errors='coerce')
    df = df[df['q-value'] <= 0.05]

    # Convert 'methRatio' and 'log10(methRatio)' columns to numeric, handling formatting issues
    df['methRatio'] = pd.to_numeric(df['methRatio'].str.replace(',', ''), errors='coerce')
    df['log10(methRatio)'] = pd.to_numeric(df['log10(methRatio)'], errors='coerce')

    # Split the dataframe based on 'methRatio'
    upwards_methylated = df[df['methRatio'] >= 1]
    downward_methylated = df[df['methRatio'] < 1]

    # Drop and recalibrate columns based on conditions
    upwards_methylated = upwards_methylated.drop(columns=['methRatio'])
    downward_methylated = downward_methylated.drop(columns=['log10(methRatio)'])
    
    upwards_methylated['methRatio'] = 10 ** upwards_methylated['log10(methRatio)']
    downward_methylated['log10(methRatio)'] = np.log10(downward_methylated['methRatio'])

    # Concatenate the adjusted dataframes
    final_df = pd.concat([upwards_methylated, downward_methylated])
    
    # Remove chromosomes not matching the specified pattern
    final_df = final_df[final_df['chrom'].str.match(r'^chr((\d{1,2})|(X|Y))$')]

    # Drop unused columns and rows with NaN in 'methRatio'
    final_df = final_df.drop(columns=['Type', 'EnsembleID'])
    final_df = final_df.dropna(subset=['methRatio'])

    # Create the 'FoldChange' column
    final_df['FoldChange'] = final_df['methRatio'].apply(lambda x: x if x >= 1 else 1/x)
    final_df = final_df[final_df['FoldChange'] >= 2]

    # Filter out rows based on the sum of ScoreX and ScoreY
    final_df = final_df.loc[(final_df['ScoreX'] + final_df['ScoreY']) >= 20]

    # Apply custom sorting function for chromosomes if available
    final_df['chromSort'] = final_df['chrom'].apply(chrom_sorter)
    final_df = final_df.sort_values(by=['chromSort', 'chromStart']).drop('chromSort', axis=1)

    # Specify and apply a new column order
    new_column_order = ['chrom', 'chromStart', 'chromEnd', 'ScoreX', 'ScoreY', 'methRatio',
                         'FoldChange', 'log10(methRatio)', 'FDR', 'q-value']
    final_df = final_df[new_column_order]

    # Construct output filename and save the processed DataFrame
    base_filename = os.path.basename(input_file)
    output_filename = f"corrected_{base_filename}"
    output_file_path = output_directory/output_filename
    final_df.to_csv(output_file_path, index=False)
    final_df = final_df.drop(columns=['FDR','q-value'])

    return final_df

# Format each file in the lists and concatanate the 2 files for each patient to a final dataframe.
df_6281_list = [format_file(file_path, cpg_and_tss_output_directory) for file_path in patient_6281]
final_df_6281 = pd.concat(df_6281_list)

df_5889_list = [format_file(file_path, cpg_and_tss_output_directory) for file_path in patient_5889]
final_df_5889 = pd.concat(df_5889_list)

In [10]:
# Define a function to detect Differentially Methylated Regions within the CpGIs and TSS sites.
def find_DMRs(df, distance_threshold, methylation_threshold):
    # Sort the input DataFrame by chromosome ('chrom') and the start position ('chromStart') of the genetic regions.
    df_sorted = df.sort_values(by=['chrom', 'chromStart'])
    DMRs = []  # Initialize an empty list to store identified DMRs.

    # Group the sorted DataFrame by chromosome and iterate over each group.
    for chrom, group in df_sorted.groupby('chrom'):
        # Initialize the first DMR with the first row's data
        current_DMR = {
            'chrom': chrom,
            'start': group.iloc[0]['chromStart'],
            'end': group.iloc[0]['chromEnd'],
            'total_log10MethRatio': group.iloc[0]['log10(methRatio)'],
            'total_FoldChange': group.iloc[0]['FoldChange'],
            'total_MethRatio': group.iloc[0]['methRatio'],
            'count': 1
        }

        # Iterate over each row in the group to assess and aggregate contiguous regions into DMRs.
        for _, row in group.iterrows():
            within_distance = row['chromStart'] <= current_DMR['start'] + distance_threshold
            avg_log10MethRatio = current_DMR['total_log10MethRatio'] / current_DMR['count']
            log10methRatio_diff = abs(avg_log10MethRatio - row['log10(methRatio)'])

            if within_distance and log10methRatio_diff <= methylation_threshold:
                # Aggregate the region into the current DMR if within distance and threshold.
                current_DMR['start'] = min(current_DMR['start'], row['chromStart'])
                current_DMR['end'] = max(current_DMR['end'], row['chromEnd'])
                current_DMR['total_log10MethRatio'] += row['log10(methRatio)']
                current_DMR['total_FoldChange'] += row['FoldChange']
                current_DMR['total_MethRatio'] += row['methRatio']
                current_DMR['count'] += 1
            else:
                # Finalize the current DMR and start a new one.
                DMRs.append([
                    current_DMR['chrom'], current_DMR['start'], current_DMR['end'], current_DMR['total_log10MethRatio'],
                    current_DMR['total_FoldChange'], current_DMR['total_MethRatio'], current_DMR['count']
                ])
                current_DMR = {
                    'chrom': chrom,
                    'start': row['chromStart'],
                    'end': row['chromEnd'],
                    'total_log10MethRatio': row['log10(methRatio)'],
                    'total_FoldChange': row['FoldChange'],
                    'total_MethRatio': row['methRatio'],
                    'count': 1
                }

        # Append the last DMR for the chromosome
        DMRs.append([
            current_DMR['chrom'], current_DMR['start'], current_DMR['end'], current_DMR['total_log10MethRatio'],
            current_DMR['total_FoldChange'], current_DMR['total_MethRatio'], current_DMR['count']
        ])

    # Convert the list of DMRs to a DataFrame and calculate additional metrics.
    DMRs_df = pd.DataFrame(DMRs, columns=['Chrom', 'Start', 'End', 'Total_log10MethRatio', 'Total_FoldChange', 'Total_MethRatio', 'Count'])
    DMRs_df['Length'] = DMRs_df['End'] - DMRs_df['Start']
    DMRs_df['Average_MethRatio'] = DMRs_df['Total_MethRatio'] / DMRs_df['Count']
    DMRs_df['Average_FoldChange'] = DMRs_df['Total_FoldChange'] / DMRs_df['Count']
    DMRs_df['Average_log10MethRatio'] = DMRs_df['Total_log10MethRatio'] / DMRs_df['Count']

    # Reorder and rename columns to match the requested format.
    DMRs_df = DMRs_df[['Chrom', 'Start', 'End', 'Length', 'Average_MethRatio', 'Average_FoldChange', 'Average_log10MethRatio']]
    DMRs_df.columns = ['chrom', 'chromStart', 'chromEnd', 'Length', 'methRatio', 'FoldChange', 'log10(methRatio)']
    DMRs_df = DMRs_df.drop(columns=['log10(methRatio)'])

    DMRs_df['log10(methRatio)'] = np.log10(DMRs_df['methRatio'])

    # Convert relevant columns to integer type.
    DMRs_df[['chromStart', 'chromEnd', 'Length']] = DMRs_df[['chromStart', 'chromEnd', 'Length']].astype(int)

    return DMRs_df

# Define a function to create a final file for each patient.
def DMR_file_creation(df, upwards_meth_filename, downwards_meth_filename, final_DMR_filename):
    # Create upwards and downwards methylated dataframes based on log10(methRatio)
    upwards_meth_df = df[df['log10(methRatio)'] >= 0]
    downwards_meth_df = df[df['log10(methRatio)'] < 0]

    # Apply the function for the DMR detection to all files
    upwards_meth_df = find_DMRs(upwards_meth_df, distance_threshold = 50000, methylation_threshold = 10)
    upwards_meth_df.to_csv(upwards_meth_filename, index = False)
 
    downwards_meth_df = find_DMRs(downwards_meth_df, distance_threshold = 50000, methylation_threshold = 10)
    downwards_meth_df.to_csv(downwards_meth_filename, index = False)
    
    # Concatanate and sort DMRs by chromosome.
    final_DMR_file = pd.concat([upwards_meth_df, downwards_meth_df])
    final_DMR_file = final_DMR_file.sort_values(by=['chrom', 'chromStart'], ignore_index= True)

    # Fit and transform the winsorized data with RobustScaler
    final_DMR_file.to_csv(final_DMR_filename, index = False)

    # Return the final DMR file
    return final_DMR_file

# Create the DMR file for both patients.
final_DMR_file_5889 = DMR_file_creation(final_df_5889, upwards_methylated_output_directory/"R_AZA_5889_upwards.csv", downwards_methylated_output_directory/"R_AZA_5889_downwards.csv", results_directory/"R_AZA_5889.csv")
final_DMR_file_6281 = DMR_file_creation(final_df_6281, upwards_methylated_output_directory/"R_AZA_6281_upwards.csv", downwards_methylated_output_directory/"R_AZA_6281_downwards.csv", results_directory/"R_AZA_6281.csv")

In [28]:
# Define the lists to store dataframes and their identifiers
AZA_non_responders = []
AZA_responders = []
DAC_non_responders = []

# List all files in the data directory
for filename in os.listdir(results_directory):
    if filename.endswith('.csv'):
        # Extract the identifier
        identifier = filename.split('_')[-1].split('.')[0]
        
        # Load the dataset
        df = pd.read_csv(os.path.join(results_directory, filename))
        
        # Determine the category of the dataset and append it to the corresponding list
        if 'NR_AZA' in filename:
            AZA_non_responders.append((identifier, df))
        elif 'R_AZA' in filename:
            AZA_responders.append((identifier, df))
        elif 'NR_DAC' in filename:
            DAC_non_responders.append((identifier, df))

plots_directory = results_directory / "Plots"

# Check if results_directory exists, if not, create it
if not os.path.exists(plots_directory):
    os.makedirs(plots_directory)

# Define a function to plot different metrics of the methRatio scores
def plot_combined_metrics(df, category, plots_directory):
    # Set up the matplotlib figure
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))  # Adjust size as needed
    
    # First plot - methRatio
    sns.histplot(df['methRatio'], bins=100, color='skyblue', kde=True, ax=axs[0, 0])
    axs[0, 0].set_title(f'Methylation ratio (methRatio) for {category}_{identifier}')
    axs[0, 0].set_xlabel('Methylation Ratio')
    axs[0, 0].set_ylabel('Frequency')
    
    # Second plot - log10(methRatio)
    sns.histplot(df['log10(methRatio)'], bins=100, color='orange', kde=True, ax=axs[0, 1])
    axs[0, 1].set_title(f'Log10(methRatio) for {category}_{identifier}')
    axs[0, 1].set_xlabel('Log10(methRatio)')
    axs[0, 1].set_ylabel('Frequency')
    
    # Adjust layout
    plt.tight_layout()
    sns.despine

    # Save the plot to file
    plot_output_path = plots_directory / f"{category}_{identifier}_combined_metrics_plot.png"
    plt.savefig(plot_output_path)
    plt.close(fig)  # Close the figure to free up memory

# Define a dictionary 'categories' with keys representing different patient categories and values being lists of tuples. 
# Each tuple contains an identifier for a patient or sample and a DataFrame ('df') with their data.
response_categories = {
    'NR_AZA': AZA_non_responders,    # Non-responders to Azacitidine treatment
    'R_AZA': AZA_responders,         # Responders to Azacitidine treatment
    'NR_DAC': DAC_non_responders    # Non-responders to Decitabine treatment
}

# Iterate over each category and its corresponding list of data tuples in the 'categories' dictionary.
for category, data_list in response_categories.items():
    # Iterate over each tuple in the data list for the current category.
    # 'identifier' is a unique identifier for the patient or sample,
    # and 'df' is the DataFrame containing their methylation data.
    for identifier, df in data_list:
        # Construct a filename for saving the plot, using the unique 'identifier'.
        # This avoids category duplication in the filename, making it more concise.
        filename = f'{identifier}'
        
        # Call the function 'plot_log10methratio' to generate and save a plot of the log10-transformed methylation ratio data contained in 'df'. 
        # The plot is saved with the constructed 'filename', and 'category' is passed to potentially customize the plot based on the patient category.
        plot_combined_metrics(df, category, plots_directory)

In [29]:
# Calculate the weighted mean log10-transformed methylation ratio for each chromosome within a DataFrame.
def calculate_weighted_mean_methRatio(df, identifier):
    # Initialize a DataFrame to store the weighted means and weighted standard errors.
    results_list = []

    # Iterate through each chromosome in the dataframe
    for chrom in df['chrom'].unique():
        # Extract the subset of the dataframe for the chromosome.
        chrom_df = df[df['chrom'] == chrom]
        
        # Split the data into two groups: above and below zero.
        above_zero = chrom_df[chrom_df['log10(methRatio)'] > 0]
        below_zero = chrom_df[chrom_df['log10(methRatio)'] < 0]
        
        # Calculate the mean of each group.
        mean_above_zero = above_zero['log10(methRatio)'].mean() if not above_zero.empty else 0
        mean_below_zero = below_zero['log10(methRatio)'].mean() if not below_zero.empty else 0
        
        # Calculate the weight (percentage) of each group.
        weight_above_zero = len(above_zero) / len(chrom_df) if not chrom_df.empty else 0
        weight_below_zero = len(below_zero) / len(chrom_df) if not chrom_df.empty else 0
        
        # Calculate the standard error for each group.
        sem_above_zero = above_zero['log10(methRatio)'].std() / np.sqrt(len(above_zero)) if not above_zero.empty else 0
        sem_below_zero = below_zero['log10(methRatio)'].std() / np.sqrt(len(below_zero)) if not below_zero.empty else 0
        
        # Calculate the weighted mean for the chromosome.
        weighted_mean = (mean_above_zero * weight_above_zero) + (mean_below_zero * weight_below_zero)
        
        # Calculate the weighted standard error.
        # Here, we assume independence between groups for simplicity.
        weighted_sem = np.sqrt((sem_above_zero ** 2 * weight_above_zero) + (sem_below_zero ** 2 * weight_below_zero))
        
        # Append the results to the list.
        results_list.append({
            'chrom': chrom,
            'weighted_mean_log10_methRatio': weighted_mean,
            'weighted_sem_log10_methRatio': weighted_sem,
            'Identifier': identifier
        })
    
    # Convert the list of dictionaries to a DataFrame.
    df_results = pd.DataFrame(results_list)
    
    return df_results

# Generate and save a bar plot visualizing mean methylation by chromosome for each patient ID within a category.
def save_patient_barplot(data, category, chrom_order, plots_directory):
    plt.figure(figsize=(14, 8))
    ax = sns.barplot(
        data=data,
        x='chrom',
        y='weighted_mean_log10_methRatio',
        hue='Identifier',  # Differentiate data by patient ID.
        errorbar='sd',  # Include standard deviation as error bars.
        palette='deep',  # Use a deep color palette for distinction.
        width=0.9,  # Bar width
        edgecolor='black',  # Set the color of the bar border.
        linewidth=0.7  # Set the width of the bar border.
    )
    
    # Apply alternating background shading for chromosomes to enhance readability.
    for i, chrom in enumerate(chrom_order):
        if i % 2 == 0:
            plt.axvspan(i - 0.5, i + 0.5, color='grey', alpha=0.2)
    
    # Set plot title, labels, and other aesthetic elements.
    plt.title(f'Median methylation by Chromosome for {category}', fontsize=16)
    ax.set_xlabel('Chromosome', fontsize=14)
    ax.set_ylabel('Median methylation', fontsize=14)
    plt.xticks(rotation=35)  # Rotate chromosome labels for clarity.
    plt.grid(True, axis='y')  # Enable grid lines along the y-axis.
    plt.legend(title='Patient ID', bbox_to_anchor=(1, 1), loc='upper left', fontsize=12)
    sns.despine()  # Remove top and right borders for a cleaner look.
    plt.tight_layout()  # Adjust layout to ensure everything fits without overlap.
    
    # Save the plot to a file, naming it uniquely based on the category and patient IDs.
    plot_output_path = plots_directory/f"{category}_weighted_mean_chrom_barplot.png"
    plt.savefig(plot_output_path)
    plt.close()

# Prepare a list and categorical data type for chromosome ordering.
chromosomes = ['chr' + str(i) for i in range(1, 23)] + ['chrX', 'chrY']
chrom_order = pd.Categorical(chromosomes, categories=chromosomes, ordered=True)

all_data = []  # Initialize a list to collect data for plotting.

# Iterate over categories and their respective data lists to calculate mean methylation ratios.
for category, data_list in response_categories.items():
    for identifier, df in data_list:
        df_weighted_mean = calculate_weighted_mean_methRatio(df, identifier)
        df_weighted_mean['Category'] = category  # Label the data with its category.
        all_data.append(df_weighted_mean)

# Concatenate all collected data into a single DataFrame for plotting.
plot_data_df = pd.concat(all_data)

# Ensure the 'chrom' column is treated as categorical with a defined order, facilitating correct plot ordering.
plot_data_df['chrom'] = plot_data_df['chrom'].astype('category')
plot_data_df['chrom'] = plot_data_df['chrom'].cat.set_categories(chromosomes, ordered=True)

# Generate and save bar plots for each category, visualizing mean methylation by chromosome.
for category in plot_data_df['Category'].unique():
    category_data = plot_data_df[plot_data_df['Category'] == category]
    save_patient_barplot(category_data, category, chromosomes, plots_directory)

In [30]:
# Define a function to create a DataFrame that contains all samples for each category and adds a category column.
def concat_dataframes_from_tuples(tuples_list, category_name):
    # Extract just the DataFrame from each tuple and add a 'category' column
    dfs = []
    for _, df in tuples_list:
        # Make a copy to avoid changing the original DataFrame
        df_copy = df.copy()
        df_copy['Category'] = category_name  # Add a new column with the category name
        dfs.append(df_copy)
        
    # Concatenate all DataFrames in the list into one, if not empty
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()  # Return an empty DataFrame if the list is empty

# Combine NR_AZA and NR_DAC into one list and rename categories
combined_non_responders = AZA_non_responders + DAC_non_responders

# Use the modified function to concatenate DataFrames and add the 'category' column
responders_df = concat_dataframes_from_tuples(AZA_responders, 'Responders')
non_responders_df = concat_dataframes_from_tuples(combined_non_responders, 'Non_Responders')

final_df = pd.concat([responders_df, non_responders_df])

In [ ]:
# Calculate the weighted means for the responders and non responders.
responder_weighted_means = calculate_weighted_mean_methRatio(responders_df, identifier="Responders")
non_responder_weighted_means = calculate_weighted_mean_methRatio(non_responders_df, identifier="Non_Responders")

# Combine the weighted means in on dataframe for plotting.
combined_weighted_means = pd.concat([responder_weighted_means, non_responder_weighted_means], ignore_index=True)
combined_weighted_means['chrom'] = pd.Categorical(combined_weighted_means['chrom'], categories=chrom_order, ordered=True)

plt.figure(figsize=(16, 8))

# Define bar width and positions
bar_width = 0.45
index = np.arange(len(chrom_order))

# Create a color map for your identifiers
color_map = {'Responders': '#008000', 'Non_Responders': '#d62728'}

# Assuming combined_weighted_means is your DataFrame
# Example DataFrame setup remains the same...

plt.figure(figsize=(16, 8))

# Define the offset from the center for each bar
# This moves the first bar to the left of the tick and the second bar to the right
offset = bar_width / 2

# Plot each group with adjusted positions
for i, identifier in enumerate(['Responders', 'Non_Responders']):
    group_df = combined_weighted_means[combined_weighted_means['Identifier'] == identifier]
    means = group_df.groupby('chrom', observed=True)['weighted_mean_log10_methRatio'].mean()
    sems = group_df.groupby('chrom', observed=True)['weighted_sem_log10_methRatio'].mean()

    # Adjust the position based on the identifier index (i)
    # This moves the bars so that one is on the left and one is on the right of the tick mark
    if i == 0:  # Responders
        pos = index - offset
    else:  # Non_Responders
        pos = index + offset

    plt.bar(pos, means.reindex(chrom_order), bar_width, yerr=sems.reindex(chrom_order), color=color_map[identifier], label=identifier, capsize=5, edgecolor='black', linewidth=0.7)

# Alternating background shading
for i, chrom in enumerate(chrom_order):
    if i % 2 == 0:
        plt.axvspan(i - 0.5, i + 0.5, color='grey', alpha=0.2)

plt.title(' ', fontsize=16)
plt.xlabel(' ', fontsize=14)
plt.ylabel('Weighted mean methylation', fontsize=16)
plt.xticks(range(len(chrom_order)), chrom_order, fontsize=14, rotation=35)
plt.yticks(fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(bbox_to_anchor=(0.93, 1), loc='upper left', fontsize=12)
sns.despine()
plt.tight_layout()
plot_output_path = plots_directory/"weighted_mean_chrom_barplot.jpeg"
plt.savefig(plot_output_path, dpi=300)

plt.show()

In [ ]:
# Copy the final dataframe in order to make modifications only the data for the violin plot
violin_plot_data = final_df
violin_plot_data['chrom'] = pd.Categorical(violin_plot_data['chrom'], categories=chrom_order, ordered=True)

# Set the size of the plot
plt.figure(figsize=(16, 8))

# Create violin plots
sns.violinplot(x='chrom', y='log10(methRatio)', hue='Category', data=violin_plot_data, split=True, inner="quart", density_norm='width', palette={'Responders': '#008000', 'Non_Responders': 'red'})
# Alternating background shading
for i, chrom in enumerate(chrom_order):
    if i % 2 == 0:
        plt.axvspan(i - 0.5, i + 0.5, color='grey', alpha=0.2)
# Improve plot aesthetics and readability
plt.title(' ',fontsize=16)
plt.xticks(fontsize=14, rotation=35)
plt.yticks(fontsize=14)
plt.xlabel(' ', fontsize=14)
plt.ylabel('Distribution of methylation', fontsize=16)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(bbox_to_anchor=(0.933, 1), loc='upper left', fontsize=12)
sns.despine() 
plot_output_path = plots_directory/"violin_plot.jpeg"
plt.tight_layout()
plt.savefig(plot_output_path, dpi=300)
# Show the plot
plt.show()

In [33]:
# Define the function to process and split dataframes
def process_and_split_df(df):
    # Drop specified columns
    df = df.drop(columns=['Length', 'methRatio', 'FoldChange'])
    
    # Split the dataframe based on the log10methratio threshold
    upmeth_df = df[df['log10(methRatio)'] >= 0 ]
    downmeth_df = df[df['log10(methRatio)'] < 0 ]
    
    return upmeth_df, downmeth_df

# Define the function to convert a dataframe to a BED file
def convert_df_to_bed(df, output_file_name):
    # Assuming the dataframe has the necessary columns for BED format in the correct order
    bed = pybedtools.BedTool.from_dataframe(df)
    bed.saveas(output_file_name)

# Process and convert dataframes in each list
def create_BED_files(dataframe_lists, output_directory):
    for list_name, dataframes in dataframe_lists.items():
        for identifier, df in dataframes:
            # Process and split the dataframe
            upmeth_df, downmeth_df = process_and_split_df(df)
            
            # Convert each split dataframe to a BED file
            upmeth_bed_file = os.path.join(output_directory, f"upwards_{list_name}_{identifier}.bed")
            convert_df_to_bed(upmeth_df, upmeth_bed_file)
            
            downmeth_bed_file = os.path.join(output_directory, f"downwards_{list_name}_{identifier}.bed")
            convert_df_to_bed(downmeth_df, downmeth_bed_file)


# Process all dataframes
create_BED_files(response_categories, bed_output_directory)

In [34]:
# At this point we will split each chromosome into bins of the same specific length. 
# Exact lengths for each human chromosome based on hg38.
chromosome_lengths = {
    'chr1': 248956422, 'chr2': 242193529, 'chr3': 198295559,
    'chr4': 190214555, 'chr5': 181538259, 'chr6': 170805979,
    'chr7': 159345973, 'chr8': 145138636, 'chr9': 138394717,
    'chr10': 133797422, 'chr11': 135086622, 'chr12': 133275309,
    'chr13': 114364328, 'chr14': 107043718, 'chr15': 101991189,
    'chr16': 90338345, 'chr17': 83257441, 'chr18': 80373285,
    'chr19': 58617616, 'chr20': 64444167, 'chr21': 46709983,
    'chr22': 50818468, 'chrX': 156040895, 'chrY': 57227415
}

# Define bin size
bin_size = 100000  # 100kb 

# Initialize an empty list to hold the bin information
bins_list = []

# Generate bins for each chromosome
for chrom, length in chromosome_lengths.items():
    num_bins = length // bin_size + (1 if length % bin_size > 0 else 0)
    for bin_num in range(num_bins):
        bin_start = bin_num * bin_size
        bin_end = min((bin_num + 1) * bin_size - 1, length)  # Ensure the last bin doesn't exceed chromosome length
        bin_name = f"{chrom}_bin{bin_num + 1}"  # Assign bin names indicating the chromosome and bin number
        bins_list.append({'chrom': chrom, 'bin_start': bin_start, 'bin_end': bin_end, 'bin_name': bin_name})

# Convert the list of bins to a DataFrame
bins_df = pd.DataFrame(bins_list)

In [35]:
# Creating an IntervalIndex for genomic bins for each chromosome for efficiency
# This dictionary comprehension creates an IntervalIndex for each chromosome found in bins_df.
# It allows for efficient interval queries, such as finding overlaps with genomic regions.
bins_intervals = {
    chrom: pd.IntervalIndex.from_arrays(
        bins_df[bins_df['chrom'] == chrom]['bin_start'],  # Start positions of bins for the chromosome
        bins_df[bins_df['chrom'] == chrom]['bin_end'],    # End positions of bins for the chromosome
        closed='both'  # Indicates that the intervals include both start and end points
    )
    for chrom in bins_df['chrom'].unique()  # Iterates over each unique chromosome in the dataframe
}

# Precompute a mapping from (chromosome, start, end) to bin name for efficient lookup
# This dictionary maps a tuple of chromosome, start, and end positions to the corresponding bin name.
# It's useful for quickly finding the name of the bin that a genomic region belongs to.
bin_name_map = {
    (row['chrom'], row['bin_start'], row['bin_end']): row['bin_name']  # Key: Tuple of chrom, start, end; Value: bin name
    for _, row in bins_df.iterrows()  # Iterates over each row in the dataframe
}

# Define a function to assign bins with maximum overlap to genomic regions in a dataframe
def assign_bins_with_max_overlap(df, bins_intervals, bin_name_map):
    # Check if the input dataframe contains the necessary columns
    if not {'chrom', 'chromStart', 'chromEnd'}.issubset(df.columns):
        raise ValueError("Input DataFrame must contain 'chrom', 'chromStart', and 'chromEnd' columns.")
    
    df['bin'] = 'No match'  # Initialize a new column for the assigned bins with a default 'No match' value

    # Process each chromosome present in the input dataframe
    for chrom in df['chrom'].unique():
        if chrom not in bins_intervals:  # Skip chromosomes not in the bins_intervals dictionary
            continue

        # Filter the input dataframe for the current chromosome
        chrom_df = df[df['chrom'] == chrom]
        # Retrieve the IntervalIndex for the current chromosome
        intervals = bins_intervals[chrom]

        # Iterate over each genomic region in chrom_df
        for index, row in chrom_df.iterrows():
            # Create an interval for the current genomic region
            region_interval = pd.Interval(left=row['chromStart'], right=row['chromEnd'], closed='both')
            # Find bins that overlap with the region_interval
            overlapping_bins = intervals.overlaps(region_interval)

            # Skip if no bins overlap with the region
            if not any(overlapping_bins):
                continue

            # Calculate overlap lengths for each overlapping bin and select the bin with maximum overlap
            overlap_lengths = [
                (min(row['chromEnd'], bin_index.right) - max(row['chromStart'], bin_index.left), bin_index.left, bin_index.right)
                for bin_index in intervals[overlapping_bins]  # Iterate over intervals that overlap with the region
            ]
            # Select the interval with the maximum overlap
            _, max_overlap_start, max_overlap_end = max(overlap_lengths, key=lambda x: x[0])
            # Lookup the bin name using the max overlapping interval's start and end
            bin = bin_name_map.get((chrom, max_overlap_start, max_overlap_end), 'No match')
            # Update the 'bin' column in the original dataframe
            df.at[index, 'bin'] = bin

    return df  # Return the modified dataframe with bins assigned

# Apply the bin detection function to each dataframe.
for category, data_tuples in response_categories.items():
    processed_dataframes = []
    for identifier, df in data_tuples:
        processed_df = assign_bins_with_max_overlap(df, bins_intervals, bin_name_map)
        processed_dataframes.append((identifier, processed_df))
    response_categories[category] = processed_dataframes

methylation_df = assign_bins_with_max_overlap(final_df, bins_intervals, bin_name_map)

In [36]:
# Define a function to perform Mann Whitney test.
def perform_mannwhitney_u_test(df, significance_levels={'***': 0.001, '**': 0.01, '*': 0.05}):
    # Identify bins with both categories present
    bins_with_both_categories = df.groupby('bin')['Category'].nunique()
    bins_with_both = bins_with_both_categories[bins_with_both_categories == 2].index.tolist()
    
    # Filter original dataset for these bins
    df_filtered = df[df['bin'].isin(bins_with_both)]
    
    # Initialize results storage
    test_results = []
    
    # Perform Mann-Whitney U tests and determine significance
    for bin_name in bins_with_both:
        data_resp = df_filtered[(df_filtered['Category'] == 'Responders') & (df_filtered['bin'] == bin_name)]['log10(methRatio)']
        data_non_resp = df_filtered[(df_filtered['Category'] == 'Non_Responders') & (df_filtered['bin'] == bin_name)]['log10(methRatio)']
        
        # Perform Mann-Whitney U test
        stat, p_value = mannwhitneyu(data_resp, data_non_resp, alternative='two-sided')
        
        # Determine significance annotation
        annotation = ''
        for symbol, threshold in significance_levels.items():
            if p_value < threshold:
                annotation = symbol
                break
        
        test_results.append((bin_name, p_value, annotation))
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(test_results, columns=['Bin', 'P-Value', 'Significance'])
    
    return results_df

# Perform the Mann Whitney U-test.
mannwhitney_results = perform_mannwhitney_u_test(final_df)


In [ ]:
# Define a function to plot the results of the Mann-Whitney test
def plot_mannwhitney_results(df, results_df, plot_output_path, category_colors='deep', top_n_bins=10):
    # Filter for significant bins and select a subset for visualization
    significant_bins = results_df[results_df['P-Value'] < 0.05].sort_values(by='P-Value', ascending=True)
    significant_bins_subset = significant_bins.head(top_n_bins)
    bins_order = significant_bins_subset['Bin'].tolist()
    
    # Filter original dataset for bins of interest
    df_significant_subset = df[df['bin'].isin(bins_order)]
    
    # Plotting with ordered bins
    plt.figure(figsize=(14, 8))
    ax = sns.boxplot(x='bin', y='log10(methRatio)', hue='Category', 
                     data=df_significant_subset, palette=category_colors, order=bins_order)
    plt.title(' ', fontsize=14)
    plt.xticks(fontsize = 12, rotation=35)
    ax.set_xlabel(' ', fontsize=14)
    ax.set_ylabel('Methylation', fontsize=14)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize= 12)
    plt.ylim(-1.5, 3.5)
    plt.grid(True, axis='y')
    
    # Add grey blocks and annotations for significance
    for i, bin in enumerate(bins_order):
        if i % 2 == 0:
            plt.axvspan(i - 0.5, i + 0.5, color='grey', alpha=0.2)
        annotation = results_df.loc[results_df['Bin'] == bin, 'Significance'].values[0]
        if annotation:
            ax.text(i, 3, annotation, ha='center', va='bottom', color='black', fontsize=18)
    
    plt.tight_layout()
    sns.despine()
    plt.savefig(plot_output_path, dpi=300)
    plt.show()

# Plot the results of the Mann Whitney U-test
mannwhitney_boxplot_output_path = plots_directory/"significant_bins_boxplot.jpeg"  # Define your output path
significant_bins = plot_mannwhitney_results(final_df, mannwhitney_results, mannwhitney_boxplot_output_path)

In [38]:
# Function to extract genomic location for a given bin
def get_genomic_location(bin_name):
    for (chrom, start, end), bin_id in bin_name_map.items():
        if bin_id == bin_name:
            return f"{chrom}:{start}-{end}"
    return "Location not found"

# Function to extract the bin name without the index number
def extract_bin_name(full_bin):
    return full_bin[full_bin.index('chr'):]

# Create a copy of the significant results
significant_results = mannwhitney_results[mannwhitney_results['P-Value'] < 0.05].copy()

# Now apply the functions to create new columns
significant_results.loc[:, 'Bin_Name'] = significant_results['Bin'].apply(extract_bin_name)
significant_results.loc[:, 'Genomic_Location'] = significant_results['Bin_Name'].apply(get_genomic_location)

# Reorder the columns for better readability
significant_results = significant_results[['Bin', 'Bin_Name', 'Genomic_Location', 'P-Value', 'Significance']]


In [39]:
# Function to extract chromosome, start, and end from Genomic_Location
def parse_genomic_location(location):
    match = re.match(r'(chr\w+):(\d+)-(\d+)', location)
    if match:
        return match.groups()
    return None, None, None

# Apply the parsing function and create new columns
significant_results[['Chromosome', 'Start', 'End']] = significant_results['Genomic_Location'].apply(parse_genomic_location).apply(pd.Series)

# Create BED file content
bed_content = significant_results[['Chromosome', 'Start', 'End', 'Bin_Name', 'P-Value', 'Significance']]

# Save to BED file
bed_file_path = 'top_bins.bed'
bed_content.to_csv(bed_file_path, sep='\t', index=False, header=False)

In [40]:
# Detect the genomic ranges of the top 10 significant bins and store the results in a pandas dataframe.
if mannwhitney_results is not None:
    mannwhitney_results.sort_values(by='P-Value', ascending=True, inplace=True)
    top_10_significant_bins = mannwhitney_results.head(10)
    top_10_significant_bins = bins_df[bins_df['bin_name'].isin(top_10_significant_bins['Bin'])]

top_10_significant_bins_df = pd.DataFrame(top_10_significant_bins)
top_10_significant_bins_df.reset_index(drop=True, inplace=True)

In [41]:
# Path to the directory containing BED files
reference_bed_path= '/Volumes/Intenso_SSD/Methylation_data/Reference_BED'

# The BED file to intersect with all others
bin_bed_file = pybedtools.BedTool('/Volumes/Intenso_SSD/Methylation_data/Reference_BED/top10_significant_bins_coordinates.bed')

# Pattern to match all BED files in the directory
bed_files_pattern = reference_bed_path + '/*.bed'

# File to save the final results
final_results_file = 'final_intersection_results.txt'

# Initialize an empty list to hold the results
all_results = []

# List all BED files in the directory
bed_files = glob.glob(bed_files_pattern)

# Iterate over each BED file in the directory
for bed_file in bed_files:
    # Skip the output.bed file itself
    if bed_file.endswith('top10_significant_bins_coordinates.bed'):
        continue
    
    # Load the current BED file
    reference_bed = pybedtools.BedTool(bed_file)
    
    # Perform the intersection
    intersection_result = bin_bed_file.intersect(reference_bed, wb=True)
    
    # Extract and store the selected columns from the intersection
    for feature in intersection_result:
        fields = str(feature).strip().split('\t')
        if len(fields) >= 8:
            all_results.append([fields[3], fields[7]])

# Save the aggregated results to the final text file
with open(final_results_file, 'w') as outfile:
    for col in all_results:
        outfile.write('\t'.join(col) + '\n')

# Structure to hold the categorized RNAs by bin
rna_bins = defaultdict(lambda: {'circRNA': [], 'miRNA': [], 'lncRNA': []})

# Open the file and process each line
with open(final_results_file, 'r') as file:
    for line in file:
        bin_id, rna_id = line.strip().split()
        if rna_id.startswith('hsa_circ'):
            rna_bins[bin_id]['circRNA'].append(rna_id)
        elif rna_id.startswith('hsa-mir') or rna_id.startswith('hsa-miR'):
            rna_bins[bin_id]['miRNA'].append(rna_id)
        else:  # Assuming the remaining are lncRNAs based on provided data structure
            rna_bins[bin_id]['lncRNA'].append(rna_id)


# Prepare the final results for each bin
final_results_data = {'Bin': [], 'circRNAs': [], 'miRNAs': [], 'lncRNAs': []}
for bin_id, details in rna_bins.items():
    final_results_data['Bin'].append(bin_id)
    final_results_data['circRNAs'].append(', '.join(details['circRNA']))
    final_results_data['miRNAs'].append(', '.join(details['miRNA']))
    final_results_data['lncRNAs'].append(', '.join(details['lncRNA']))

rna_interactions_df = pd.DataFrame(final_results_data)


In [ ]:
#Set chromosomes
circle = Gcircle(figsize=(14,14)) 
with open("/Volumes/Intenso_SSD/Methylation_data/general_chromosomes_human.csv") as f:
    f.readline()
    for line in f:
        line   = line.rstrip().split(",") 
        name   = line[0]
        length = int(line[-1]) 
        arc    = Garc(arc_id=name, size=length, interspace=2, raxis_range=(735,785), labelposition=80, label_visible=True, labelsize=16)
        circle.add_garc(arc) 

circle.set_garcs(0,360) 
for arc_id in circle.garc_dict:
    circle.tickplot(arc_id, raxis_range=(785,800), tickinterval=20000000, ticklabels=None) 

#cytoband

color_dict   = {"gneg":"#FFFFFF00", "gpos25":"#EEEEEE", "gpos50":"#BBBBBB", "gpos75":"#777777", "gpos100":"#000000", "gvar":"#FFFFFF00", "stalk":"#C01E27", 
               "acen":"#D82322"}

arcdata_dict = defaultdict(dict)
with open("/Volumes/Intenso_SSD/Methylation_data/karyotype_human.csv") as f:
    f.readline()
    for line in f:
        line  = line.rstrip().split(",")
        name  = line[0]     
        start = int(line[1])-1 
        width = int(line[2])-(int(line[1])-1) 
        if name not in arcdata_dict:
            arcdata_dict[name]["positions"] = []
            arcdata_dict[name]["widths"]    = [] 
            arcdata_dict[name]["colors"]    = [] 
        arcdata_dict[name]["positions"].append(start) 
        arcdata_dict[name]["widths"].append(width)
        arcdata_dict[name]["colors"].append(color_dict[line[-1]])

for key in arcdata_dict:
    circle.barplot(key, data=[1]*len(arcdata_dict[key]["positions"]), positions=arcdata_dict[key]["positions"], 
                   width=arcdata_dict[key]["widths"], raxis_range=[735,785], facecolor=arcdata_dict[key]["colors"]) 

df = methylation_df

# Load seaborn deep palette
palette = sns.color_palette("deep")
responders_color_pos = palette[0]  # Deep blue for positive values
responders_color_neg = sns.light_palette(responders_color_pos, n_colors=3)[1]  # Lighter blue for negative values
non_responders_color_pos = palette[1]  # Deep red for positive values
non_responders_color_neg = sns.light_palette(non_responders_color_pos, n_colors=3)[1]  # Lighter red for negative values

# Filter data for responders and non-responders
responders = df[df['Category'] == 'Responders']
non_responders = df[df['Category'] == 'Non_Responders']

# Function to add bar plots for positive and negative values separately
def add_barplots(data, chrom, radial_range_pos, radial_range_neg, color_pos, color_neg):
    positions = [(start + end) // 2 for start, end, _ in data]
    meth_ratios = [log10_meth_ratio for _, _, log10_meth_ratio in data]
    widths = [end - start for start, end, _ in data]
    
    # Separate positive and negative log10(methRatio) values
    pos_data = [val if val >= 0 else 0 for val in meth_ratios]
    neg_data = [-val if val < 0 else 0 for val in meth_ratios]
    
    # Plot positive values (hypermethylated)
    circle.barplot(chrom, data=pos_data, positions=positions, width=widths, raxis_range=radial_range_pos, facecolor=color_pos, edgecolor=color_pos, linewidth=0.05)
    
    # Plot negative values (hypomethylated)
    circle.barplot(chrom, data=neg_data, positions=positions, width=widths, raxis_range=radial_range_neg, facecolor=color_neg, edgecolor=color_neg, linewidth=0.05)

# Prepare data for responders
responder_data = defaultdict(list)
for _, row in responders.iterrows():
    chrom = row['chrom']
    start = row['chromStart']
    end = row['chromEnd']
    log10_meth_ratio = row['log10(methRatio)']
    responder_data[chrom].append((start, end, log10_meth_ratio))

# Prepare data for non-responders
non_responder_data = defaultdict(list)
for _, row in non_responders.iterrows():
    chrom = row['chrom']
    start = row['chromStart']
    end = row['chromEnd']
    log10_meth_ratio = row['log10(methRatio)']
    non_responder_data[chrom].append((start, end, log10_meth_ratio))

# Add bar plots for responders
for chrom, values in responder_data.items():
    add_barplots(values, chrom, radial_range_pos=(650, 730), radial_range_neg=(580, 640), color_pos='blue', color_neg=responders_color_neg)

# Add bar plots for non-responders
for chrom, values in non_responder_data.items():
    add_barplots(values, chrom, radial_range_pos=(510, 570), radial_range_neg=(440, 500), color_pos='orange', color_neg=non_responders_color_neg)   

# Add label to the center of the circos plot
plt.text(0.5, 0.5, 'Genome Wide Methylation Profile of \n Responders (blue) and Non Responders (orange)', horizontalalignment='center', verticalalignment='center', 
         transform=circle.figure.transFigure, fontsize=15)

# Show the final plot
circle.figure.savefig('circos.png', dpi=300)